In [367]:
# Loading of libraries
import numpy as np
import scipy.spatial.distance
import matplotlib.pyplot as plt

%matplotlib inline

In [368]:
# Design variables
AverageRadius            = 1
DeviationRadius          = AverageRadius / 10
ParticleCount            = 20
Viscosity                = 1E3
ViscosityR               = 32 / 3 * Viscosity
RotationalViscoisty      = 1E3
RotationalViscoistyR     = 4 * np.pi * RotationalViscoisty
NeighbourDistance        = 2.7 * AverageRadius
SpringConstant           = 1
Chi                      = 1         # Rotational viscosity    - 1 in natural units
Zeta                     = 1         # Translational visocsity - 1 in natural units
k                        = 1         # Sprint constant         - 1 in natural units

# Simulation variables
TimeStep                 = 0.01
L_align                  = 0.1       # Range from 0.1 to 1
L_noise                   = 0.04      # Range from 0.04 to 0.08
L_F_in                   = 0.3
L_T_in                   = 3
L_self                   = 0.03
T_align                  = L_align * k * Chi / Zeta
T_noise                  = np.sqrt(L_noise / Zeta / TimeStep * 2 * k * Chi**2)
F_in                     = L_F_in * AverageRadius * k
T_in                     = L_T_in * Chi * k / Zeta
F_self                   = L_self * k * AverageRadius


# Initiate the pengiuins
Radius                   = np.random.normal(AverageRadius, DeviationRadius, ParticleCount)
Location                 = (np.vstack((
                             np.tile(np.arange(0,10), ParticleCount / 10) * 1,
                             np.repeat(np.arange(0,ParticleCount / 10), 10) * 1
                           ))).T# + np.random.normal(0,1,(ParticleCount,2))
Velocity                 = np.zeros([ParticleCount, 2])
Orientation              = np.ones(ParticleCount) * np.pi / 2 + np.random.uniform(-pi/4, pi/4, ParticleCount)
AngularVelocity          = np.zeros(ParticleCount)


In [450]:
## This function calculates the force on the particles:
# ParticleRadius  = Radius of the particle
# ViscosityR      = Scaled viscosity of the system
# Velocity        = Velocity of the particle
# NeighbourRadius = Radius in which Neighbours can be found
def ParticleForce(ParticleRadius, Orientation, NeighbourRadius):
    # Particle force = Self-Propulsion + Boundary + Repulsion
    Heaviside       = piecewise(NeighbourRadius,
                          [NeighbourRadius < pi, NeighbourRadius > np.pi],
                          [0, 1])
    PropulsionForce = F_self * Orientation
    BoundaryForce   = (NeighbourRadius - np.pi) * F_in * Heaviside * Orientation
    RepulsionForce  = SpringConstant * sum(NeighbourOverlap)
    
    return PropulsionForce + BoundaryForce + RepulsionForce

    
def ParticleTorque(ParticleRadius, RotationalViscoistyR, Rotation):
    Heaviside      = piecewise(NeighbourRadius,
                          [NeighbourRadius < pi, NeighbourRadius > np.pi],
                          [0, 1])
    # Particletorque = Self-Torque + Noise + Align
    BoundaryToruqe = T_in * deltaTheta * Heaviside
    NoiseTorque    = T_noise * random(-1,1,1)
    AlignTorque    = T_align * sum(Theta_neighbour_mismatch)
    
    return BoundaryTorque + NoiseTorque + AlignTorque


## This functions creates a list of neighbours for all particles
#  ParticleLocations is the Nx2 array containing the locations of the particles
def NeighbourList(Location):
    Distances = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(Location))
    Neighbours = np.nonzero(Distances + np.diag(np.ones(Location.shape[0]) * 3) < 2.7);
    return np.array(Neighbours, dtype='int64').T
    
def NeighbourRadius(Location, Neighbours):
    Angles = np.arctan2(
        Location[Neighbours[:,0],1] - Location[Neighbours[:,1],1],
        Location[Neighbours[:,0],0] - Location[Neighbours[:,1],0],
    )
    AvgAngle = np.zeros([Location.shape[0]])
    for i in range(0, Location.shape[0]):
        N = Neighbours[:,0] == i
        A = np.array(np.nonzero(N)[0])
        AvgAngle[i] = np.sum(Angles[A]) / np.sum(N)
    return AvgAngle

def NeighbourOverlap(Location, Neighbours, Radius):
    Overlap = np.zeros([Location.shape[0]])
    for i in range(0,Location.shape[0]):
        N = Neighbours[:,0] == i
        Neigh = Neighbours[np.nonzero(N), 1];
        d = np.sqrt(np.sum(np.power(Location[i,:] - Location[Neigh, :],2 ), axis=2));
        # Overlap function between two circles
        r = Radius[Neigh]
        R = Radius[i]
        A = r**2 * np.arccos((d**2 + r**2 - R**2) / (2*d*r))
        A = A + R**2 * np.arccos((d**2 + R**2 - r**2) / (2*d*R))
        A = A - np.sqrt((-d+r+R)*(d+r-R)*(d-r+R)*(d+r+R))
        Overlap[i] = np.sum(np.nan_to_num(A));
    return Overlap

In [452]:
Neighbours = NeighbourList(Location)
AvgAngles  = NeighbourRadius(Location, Neighbours);
Overlap    = NeighbourOverlap(Location, Neighbours, Radius);
print(Overlap)

max_steps = np.int(1E3);
for step in range(0,max_steps):
    Velocity = Velocity;

[-0.37925384 -0.1778022   0.40730623  0.23745201 -0.73732616 -0.70733744
  0.0365601   0.22992295 -0.37756057 -0.0275624  -0.33013887 -0.74500468
  0.44687703 -0.30876875  0.10450234 -1.42638906  0.5510183  -0.16712297
  0.37917517 -0.45612383]


C:\Users\sybre\Anaconda3\lib\site-packages\ipykernel\__main__.py:58: RuntimeWarning: invalid value encountered in arccos
C:\Users\sybre\Anaconda3\lib\site-packages\ipykernel\__main__.py:59: RuntimeWarning: invalid value encountered in arccos
C:\Users\sybre\Anaconda3\lib\site-packages\ipykernel\__main__.py:60: RuntimeWarning: invalid value encountered in sqrt
